In [ ]:
!git clone https://github.com/ma2za/emotion-classification.git

In [ ]:
!mv emotion-classification/emotion_classification/src/roberta_emotion roberta_emotion

In [ ]:
!pip install -q transformers datasets evaluate wandb

In [ ]:
import torch
import wandb
from datasets import load_dataset
from evaluate import evaluator
from huggingface_hub import notebook_login
from sklearn.metrics import accuracy_score, f1_score
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers.data.data_collator import default_data_collator
from transformers.optimization import get_linear_schedule_with_warmup

In [ ]:
from roberta_emotion.modeling_roberta_emotion import RobertaEmotion
from roberta_emotion.configuration_roberta_emotion import RobertaEmotionConfig

In [ ]:
%env WANDB_PROJECT=emotion_classifier

In [ ]:
wandb.login()

In [ ]:
notebook_login()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
def tokenization(sample):
    return tokenizer(sample["text"], padding=True, truncation=True)

## Dataset

In [ ]:
dataset = load_dataset("emotion")

In [ ]:
dataset = dataset.map(tokenization, batched=True, batch_size=None)

In [ ]:
dataset.set_format("torch", columns=["input_ids", "label"])

In [ ]:
id2label =  {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
  }

label2id = {
    "sadness": 0,
    "joy": 1,
    "love": 2,
    "anger": 3,
    "fear": 4,
    "surprise": 5
  }

In [ ]:
train_dataset = dataset["train"]
train_dataset.remove_columns(["text"])

valid_dataset = dataset["validation"]
valid_dataset.remove_columns(["text"])

In [ ]:
train_loader = DataLoader(
            train_dataset,
            batch_size=64,
            collate_fn=default_data_collator,
            drop_last=False,
            num_workers=0,
            pin_memory=True
        )

valid_loader = DataLoader(
            valid_dataset,
            batch_size=64,
            collate_fn=default_data_collator,
            drop_last=False,
            num_workers=0,
            pin_memory=True
        )

## Model

In [ ]:
RobertaEmotionConfig.register_for_auto_class()

In [ ]:
RobertaEmotion.register_for_auto_class("AutoModel")

In [ ]:
config = RobertaEmotionConfig(id2label = id2label, 
                              label2id = label2id, 
                              hidden_size = 768,
                              num_labels = 6)

In [ ]:
model = RobertaEmotion(config).to(device)

## Training

In [ ]:
model.backbone.requires_grad = False

In [ ]:
optimizer = AdamW(model.parameters(),lr= 2e-05, betas= (0.9, 0.999), eps= 1e-08)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=2000)

In [ ]:
def compute_metrics(preds, labels):
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return acc , f1

In [ ]:
def evaluation(dataloader):
    model.eval()
    total_samples = 0
    total_loss = 0
    total_acc = 0
    total_f1 = 0
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, labels=labels)
        acc, f1 = compute_metrics(outputs.logits.argmax(-1).detach().cpu(), labels.detach().cpu())
        total_acc += acc*len(labels)
        total_f1 += f1*len(labels)
        total_samples += len(labels)
        total_loss += outputs.loss.detach()*len(labels)
    return total_acc/total_samples, total_f1/total_samples, total_loss/total_samples

In [ ]:
best_f1 = 0

wandb.init(project="emotion_classifier")

for epoch in range(8):
    model.train()
    for step, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        model.zero_grad()
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, labels=labels)
        outputs.loss.backward()

        optimizer.step()
        lr_scheduler.step()
    valid_acc, valid_f1, valid_loss = evaluation(valid_loader)
    wandb.log({"eval/loss": valid_loss, "eval/f1": valid_f1, "eval/accuracy": valid_acc})

    if best_f1 < valid_f1:
        best_f1 = valid_f1
        torch.save(model.state_dict(), "pytorch_model.bin")

wandb.finish()

In [ ]:
model.push_to_hub("roberta-emotion")
tokenizer.push_to_hub("roberta-emotion")

## Evaluation

In [ ]:
task_evaluator = evaluator("text-classification")

In [ ]:
results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data="emotion",
    subset="split",
    split="validation",
    metric="accuracy",
    label_mapping=label2id,
    strategy="bootstrap",
    n_resamples=10,
    random_state=0
)

results